In [1]:
!pip install plotly.express

Column Definations available here - 
https://support.google.com/analytics/answer/7029846#zippy=

In [51]:
import pandas as pd
pd.set_option('display.max_columns', None)
from google.cloud import bigquery
client = bigquery.Client()

In [75]:
#Python Helper Functions 


def get_df_from_query(query):
    # return df from the query given. 
    #Dont pass on big data queries. Usefule for small datasets  
    client = bigquery.Client()
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df

def get_table_detail_dict(project_id,dataset_id,table_name=None):
    table_details_dict = {}
    table_details_dict['project_id'] = project_id 
    table_details_dict['dataset_id'] = dataset_id
    table_details_dict['table_name'] = table_name
    return table_details_dict

def get_datatypes_of_column(table_details_dict):
    query = """SELECT 
    column_name,ordinal_position,
    is_nullable,data_type
    FROM `{project_id}.{dataset_id}.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name = {table_name}
    """.format(project_id = table_details_dict['project_id'],
               dataset_id = table_details_dict['dataset_id'],
               table_name = table_details_dict['table_name'])
    return [get_df_from_query(query),query]

def get_all_tables_in_dataset(table_details_dict):
    query = """
    SELECT TABLE_NAME from `{project_id}.{dataset_id}.INFORMATION_SCHEMA.TABLES`
    """.format(project_id = table_details_dict['project_id'],
               dataset_id = table_details_dict['dataset_id'])
    return [get_df_from_query(query),query]

def get_topN_data_from_table(N,table_details_dict, all_tables=True):
    if all_tables:
        query = """
        SELECT *
        FROM 
            `{project_id}.{dataset_id}.events_*`
        LIMIT {N}
        """.format(project_id = table_details_dict['project_id'],
                   dataset_id = table_details_dict['dataset_id'],
                   N = N)
        return [get_df_from_query(query),query]

In [68]:
project_id='bigquery-public-data'
dataset_id= 'ga4_obfuscated_sample_ecommerce'

In [77]:
table_params = get_table_detail_dict(project_id,dataset_id)
get_topN_data_from_table(2, table_params,all_tables=True)[0]

,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,privacy_info,user_properties,user_first_touch_timestamp,user_ltv,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items
0,20210113,1610497831308035,page_view,"[{'key': 'session_engaged', 'value': {'string_...",NaN,NaN,1633042825,NaN,None,1004545.8016952581,"{'analytics_storage': None, 'ads_storage': Non...",[],1610497826248906,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
1,20210113,1610497826248906,session_start,"[{'key': 'engaged_session_event', 'value': {'s...",NaN,NaN,-1239046436,NaN,None,1004545.8016952581,"{'analytics_storage': None, 'ads_storage': Non...",[],1610497826248906,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': '(none)', 'name': '(direct)', 'sour...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]


In [78]:
get_topN_data_from_table(2, table_params,all_tables=True)[1]

'\n        SELECT *\n        FROM \n            `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`\n        LIMIT 2\n        '

In [5]:
%%bigquery
# Data which we will be working with. 2 Months Ecommerce website(google merchandise) data for Google Analytics
SELECT *
FROM 
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
LIMIT 2

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.86rows/s]


,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,user_ltv,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items
0,20201121,1605946429580542,page_view,"[{'key': 'all_data', 'value': {'string_value':...",NaN,NaN,-2804281214,NaN,None,1011144.1893306037,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
1,20201121,1605946429580542,first_visit,"[{'key': 'page_location', 'value': {'string_va...",NaN,NaN,-2804281214,NaN,None,1011144.1893306037,...,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Americas', 'sub_continent': 'No...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]


In [6]:
%%bigquery
# GET ALL Tabels in Dataset 
SELECT TABLE_NAME from `bigquery-public-data.ga4_obfuscated_sample_ecommerce.INFORMATION_SCHEMA.TABLES`
    
#reference: https://cloud.google.com/bigquery/docs/information-schema-tables 

Downloading: 100%|██████████| 92/92 [00:00<00:00, 94.69rows/s] 


,TABLE_NAME
0,events_20210109
1,events_20201109
2,events_20210110
3,events_20210123
4,events_20201107
...,...
87,events_20210116
88,events_20201119
89,events_20201125
90,events_20201118


In [7]:
%%bigquery
# Get all column Name in a given table 
SELECT column_name
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'events_20210116'

Downloading: 100%|██████████| 23/23 [00:01<00:00, 21.26rows/s]


,column_name
0,event_date
1,event_timestamp
2,event_name
3,event_params
4,event_previous_timestamp
5,event_value_in_usd
6,event_bundle_sequence_id
7,event_server_timestamp_offset
8,user_id
9,user_pseudo_id


In [11]:
%%bigquery
# Column Value Description 
SELECT 
    column_name,ordinal_position,
    is_nullable,data_type
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'events_20210116'

Downloading: 100%|██████████| 23/23 [00:01<00:00, 21.68rows/s]


,column_name,ordinal_position,is_nullable,data_type
0,event_date,1,YES,STRING
1,event_timestamp,2,YES,INT64
2,event_name,3,YES,STRING
3,event_params,4,NO,"ARRAY<STRUCT<key STRING, value STRUCT<string_v..."
4,event_previous_timestamp,5,YES,INT64
5,event_value_in_usd,6,YES,FLOAT64
6,event_bundle_sequence_id,7,YES,INT64
7,event_server_timestamp_offset,8,YES,INT64
8,user_id,9,YES,STRING
9,user_pseudo_id,10,YES,STRING


In [12]:
%%bigquery
# All the ARRAY Structure Columns
SELECT 
    column_name,ordinal_position,
    is_nullable,data_type
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'events_20210116'
AND data_type like 'ARRAY%'

Downloading: 100%|██████████| 3/3 [00:01<00:00,  2.87rows/s]


,column_name,ordinal_position,is_nullable,data_type
0,event_params,4,NO,"ARRAY<STRUCT<key STRING, value STRUCT<string_v..."
1,user_properties,12,NO,"ARRAY<STRUCT<key INT64, value STRUCT<string_va..."
2,items,23,NO,"ARRAY<STRUCT<item_id STRING, item_name STRING,..."


In [41]:
%%bigquery
SELECT event_params,user_properties,items
FROM 
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.86rows/s]


,event_params,user_properties,items
0,"[{'key': 'session_engaged', 'value': {'string_...",[],[]
1,"[{'key': 'page_title', 'value': {'string_value...",[],[]
2,"[{'key': 'ga_session_number', 'value': {'strin...",[],[]
3,"[{'key': 'debug_mode', 'value': {'string_value...",[],[]
4,"[{'key': 'session_engaged', 'value': {'string_...",[],[]


In [13]:
%%bigquery
# All the Struct Structure Columns
SELECT 
    column_name,ordinal_position,
    is_nullable,data_type
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'events_20210116'
AND data_type like 'STRUCT%'

Downloading: 100%|██████████| 8/8 [00:00<00:00,  8.18rows/s]


,column_name,ordinal_position,is_nullable,data_type
0,privacy_info,11,YES,"STRUCT<analytics_storage INT64, ads_storage IN..."
1,user_ltv,14,YES,"STRUCT<revenue FLOAT64, currency STRING>"
2,device,15,YES,"STRUCT<category STRING, mobile_brand_name STRI..."
3,geo,16,YES,"STRUCT<continent STRING, sub_continent STRING,..."
4,app_info,17,YES,"STRUCT<id STRING, version STRING, install_stor..."
5,traffic_source,18,YES,"STRUCT<medium STRING, name STRING, source STRING>"
6,event_dimensions,21,YES,STRUCT<hostname STRING>
7,ecommerce,22,YES,"STRUCT<total_item_quantity INT64, purchase_rev..."


In [40]:
%%bigquery
SELECT       
    privacy_info,user_ltv,device,
    geo,app_info,app_info,traffic_source,
    event_dimensions,ecommerce
FROM 
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
LIMIT 2

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.69rows/s]


,privacy_info,user_ltv,device,geo,app_info,app_info_1,traffic_source,event_dimensions,ecommerce
0,"{'analytics_storage': None, 'ads_storage': Non...","{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Southe...",None,None,"{'medium': 'organic', 'name': '(organic)', 'so...",None,"{'total_item_quantity': None, 'purchase_revenu..."
1,"{'analytics_storage': None, 'ads_storage': Non...","{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Asia', 'sub_continent': 'Southe...",None,None,"{'medium': 'organic', 'name': '(organic)', 'so...",None,"{'total_item_quantity': None, 'purchase_revenu..."


In [44]:
%%bigquery
SELECT 
    privacy_info.key as key
FROM 
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`,
    UNNEST(privacy_info) as privacy_info
LIMIT 2

Executing query with job ID: f379f3bf-da2c-4268-8506-4c061cabea92
Query executing: 0.33s


ERROR:
 400 Values referenced in UNNEST must be arrays. UNNEST contains expression of type STRUCT<analytics_storage INT64, ads_storage INT64, uses_transient_token STRING> at [5:12]

(job ID: f379f3bf-da2c-4268-8506-4c061cabea92)

                     -----Query Job SQL Follows-----                     

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:SELECT 
   2:    privacy_info.key as key
   3:FROM 
   4:    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`,
   5:    UNNEST(privacy_info) as privacy_info
   6:LIMIT 2
    |    .    |    .    |    .    |    .    |    .    |    .    |


In [14]:
%%bigquery
# All the STRING, INT64 Columns
SELECT 
    column_name,ordinal_position,
    is_nullable,data_type
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'events_20210116'
AND data_type not like 'STRUCT%'
AND data_type not like 'ARRAY%'

Downloading: 100%|██████████| 12/12 [00:01<00:00, 11.27rows/s]


,column_name,ordinal_position,is_nullable,data_type
0,event_date,1,YES,STRING
1,event_timestamp,2,YES,INT64
2,event_name,3,YES,STRING
3,event_previous_timestamp,5,YES,INT64
4,event_value_in_usd,6,YES,FLOAT64
5,event_bundle_sequence_id,7,YES,INT64
6,event_server_timestamp_offset,8,YES,INT64
7,user_id,9,YES,STRING
8,user_pseudo_id,10,YES,STRING
9,user_first_touch_timestamp,13,YES,INT64


In [53]:
query = """
SELECT 
    column_name,ordinal_position,
    is_nullable,data_type
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'events_20210116'
"""
df = get_df_from_query(query)

In [54]:
df.head()

,column_name,ordinal_position,is_nullable,data_type
0,event_date,1,YES,STRING
1,event_timestamp,2,YES,INT64
2,event_name,3,YES,STRING
3,event_params,4,NO,"ARRAY<STRUCT<key STRING, value STRUCT<string_v..."
4,event_previous_timestamp,5,YES,INT64


In [55]:
df[df['data_type'].str.startswith('ARRAY', na=False)]

,column_name,ordinal_position,is_nullable,data_type
3,event_params,4,NO,"ARRAY<STRUCT<key STRING, value STRUCT<string_v..."
11,user_properties,12,NO,"ARRAY<STRUCT<key INT64, value STRUCT<string_va..."
22,items,23,NO,"ARRAY<STRUCT<item_id STRING, item_name STRING,..."


In [64]:
df[df['column_name']=='event_params']['data_type'].iloc[0]

'ARRAY<STRUCT<key STRING, value STRUCT<string_value STRING, int_value INT64, float_value FLOAT64, double_value FLOAT64>>>'

In [65]:
df[df['column_name']=='event_params']['data_type'].iloc[0].split(' ')

['ARRAY<STRUCT<key',
 'STRING,',
 'value',
 'STRUCT<string_value',
 'STRING,',
 'int_value',
 'INT64,',
 'float_value',
 'FLOAT64,',
 'double_value',
 'FLOAT64>>>']